In [0]:
dbutils.fs.cp("file:/Workspace/Shared/sales_data1.csv", "dbfs:/FileStore/sales_data1.csv")

True

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/customer_data.json", "dbfs:/FileStore/customer_data.json")

True

In [0]:
dbutils.fs.cp("file:/Workspace/Shared/new_sales_data.csv", "dbfs:/FileStore/new_sales_data.csv")

True

In [0]:
# 1.1 Load sales_data.csv into a DataFrame and Write as Delta Table
# Load sales_data.csv into DataFrame
sales_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/sales_data1.csv")

In [0]:

# Write the DataFrame as Delta Table
sales_df.write.format("delta").mode("overwrite").save("/delta/sales_data")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DateType

# Define schema for customer_data.json
customer_schema = StructType([
    StructField("CustomerID", StringType(), True),
    StructField("CustomerName", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("SignupDate", DateType(), True)  
])


In [0]:
# 1.2 Load customer_data.json into a DataFrame and Write as Delta Table
# Load the JSON data with the defined schema
customer_df = spark.read.format("json").schema(customer_schema).load("dbfs:/FileStore/customer_data.json")
customer_df.show()

+----------+-------------+------+----------+
|CustomerID| CustomerName|Region|SignupDate|
+----------+-------------+------+----------+
|      C001|     John Doe| North|2022-07-01|
|      C002|   Jane Smith| South|2023-02-15|
|      C003|Emily Johnson|  East|2021-11-20|
|      C004|Michael Brown|  West|2022-12-05|
|      C005|  Linda Davis| North|2023-03-10|
+----------+-------------+------+----------+



In [0]:
# Write the DataFrame as Delta Table
customer_df.write.format("delta").mode("overwrite").save("/delta/customer_data")

In [0]:
# 2.1 Load new_sales_data.csv into a DataFrame and Write as Delta Table
# Load new_sales_data.csv into DataFrame
new_sales_df = spark.read.format("csv").option("header", "true").load("dbfs:/FileStore/new_sales_data.csv")

In [0]:
# Write the new DataFrame as Delta Table
new_sales_df.write.format("delta").mode("overwrite").save("/delta/new_sales_data")

In [0]:
from delta.tables import *

# Load existing Delta table
delta_table =spark.read.format("delta").load("/delta/sales_data")
delta_new_sales=spark.read.format("delta").load("/delta/new_sales_data")
# Create temporary views for SQL operations
delta_table.createOrReplaceTempView("delta_sales_data")
delta_new_sales.createOrReplaceTempView("new_sales_data")

In [0]:
# Merge new sales data into existing Delta Table
spark.sql("""
          MERGE INTO delta_sales_data AS target
          USING new_sales_data AS source
          ON target.OrderID = source.OrderID
          WHEN MATCHED THEN UPDATE SET target.OrderDate=source.OrderDate, target.CustomerID = source.CustomerID, target.Product=source.Product,
          target.Quantity=source.Quantity, target.Price=source.Price
          WHEN NOT MATCHED THEN INSERT (OrderID,OrderDate,CustomerID,Product,Quantity,Price)
          VALUES (source.OrderID, source.OrderDate, source.CustomerID, source.Product, source.Quantity, source.Price)
""")


DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql("SELECT * FROM delta_sales_data").show()

+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|      10|15.75|
|   1009|2024-01-22|      C006|Widget E|      14|20.00|
|   1010|2024-01-23|      C007|Widget F|       6|35.00|
+-------+----------+----------+--------+--------+-----+



In [0]:
# Register delta table
spark.sql("CREATE TABLE IF NOT EXISTS delta_sales_table USING DELTA LOCATION '/delta/sales_data'")

DataFrame[]

In [0]:
# Check the schema of the Delta table
spark.read.format("delta").load("/delta/sales_data").printSchema()


root
 |-- OrderID: string (nullable = true)
 |-- OrderDate: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Price: string (nullable = true)



In [0]:
# 3. Optimize Delta Table
# 3.1 Apply the OPTIMIZE command on the Delta Table and use Z-Ordering on an appropriate column.
spark.sql(" OPTIMIZE delta_sales_table ")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [0]:
spark.sql("ALTER TABLE delta_sales_table ADD COLUMN CustomerID STRING")

DataFrame[]

In [0]:
spark.sql("OPTIMIZE delta_sales_table ZORDER BY (CustomerID)")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,totalClusterParallelism:bigint,totalScheduledTasks:bigint,autoCompactParallelismStats:struct<maxClusterActiveParallelism:bigint,minClusterActiveParallelism:bigint,maxSessionActiveParallelism:bigint,minSessionActiveParallelism:bigint>,de

In [0]:
# 4. Advanced Features
# 1. Use DESCRIBE HISTORY to inspect the history of changes for a Delta Table.
spark.sql("DESCRIBE HISTORY delta_sales_table").show(truncate=False)

+-------+-------------------+----------------+----------------------------------+-----------+---------------------------------------------------------------------------------------------+----+------------------+--------------------+-----------+-----------------+-------------+-----------------------------------------------------------+------------+------------------------------------------+
|version|timestamp          |userId          |userName                          |operation  |operationParameters                                                                          |job |notebook          |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                           |userMetadata|engineInfo                                |
+-------+-------------------+----------------+----------------------------------+-----------+---------------------------------------------------------------------------------------------+----+------------------+---

In [0]:
# 2. Use VACUUM to remove old files from the Delta Table.
spark.sql("""
         VACUUM delta_sales_table RETAIN 168 HOURS
          """)

DataFrame[path: string]

In [0]:
# 5. Hands-on Exercises
# 1. Using Delta Lake for Data Versioning
# Query historical versions of the Delta Table using Time Travel.
historical_df = spark.read.format("delta").option("versionAsOf", 1).load("/delta/sales_data")
historical_df.show()


+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|2024-01-15|      C001|Widget A|      10|25.50|
|   1003|2024-01-16|      C001|Widget C|       8|22.50|
|   1004|2024-01-17|      C003|Widget A|      15|25.50|
|   1005|2024-01-18|      C004|Widget D|       7|30.00|
|   1006|2024-01-19|      C002|Widget B|       9|15.75|
|   1007|2024-01-20|      C005|Widget C|      12|22.50|
|   1008|2024-01-21|      C003|Widget A|      10|25.50|
|   1002|2024-01-16|      C002|Widget B|      10|15.75|
|   1009|2024-01-22|      C006|Widget E|      14|20.00|
|   1010|2024-01-23|      C007|Widget F|       6|35.00|
+-------+----------+----------+--------+--------+-----+



In [0]:
# 2. Building a Reliable Data Lake with Delta Lake:
# Implement schema enforcement and handle data updates with Delta Lake 
# Optimize data layout and perform vacuum operations to maintain storage efficiency.
new_sales_df.write.format("delta").mode("append").option("mergeSchema", "true").save("/path/to/delta/sales_data")
spark.sql("VACUUM delta_sales_table")

DataFrame[path: string]